# Analysis of Automobile Inventory Data

## Copyright (c) 2018, Faststream Technologies
## Author: Sudhanva Narayana

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report

### CURR and PARENT directory constants

In [58]:
CURR_DIR = os.path.dirname(os.path.abspath('__file__'))
PARENT_DIR = os.path.abspath(os.path.join(CURR_DIR, os.pardir))

### Import dataset ignoring headers

In [59]:
df = pd.read_excel(PARENT_DIR + '\\assets\\data\\items_analysis.xlsx', sheet_name='Sheet1')

### Dataset

In [60]:
df.head()

,Item_Number,Make_Or_Buy_Flag,Minimum_Order_Quantity,Standard_Cost,Ordered_Quantity,Month,Request_Date,Cumulative LT
0,066.01.374.1.37,Make,NaN,1027.19182,2,2015-08,2015-08-24 00:00:00,76.986458
1,066.322.002,Buy,5.0,893.80000,2,2015-01,2015-01-23 00:00:00,156.000000
2,066.322.002,Buy,5.0,893.80000,2,2015-01,2014-12-22 15:09:00,156.000000
3,066.322.002,Buy,5.0,893.80000,4,2015-09,2015-09-16 00:00:00,156.000000
4,066.322.002,Buy,5.0,893.80000,2,2015-02,2015-01-28 00:00:00,156.000000


In [61]:
# Convert uppercase to lowercase and replace empty spaces with underscore
df.columns = [i.replace(" ", "_").lower() for i in df.columns]
df.columns

Index(['item_number', 'make_or_buy_flag', 'minimum_order_quantity',
       'standard_cost', 'ordered_quantity', 'month', 'request_date',
       'cumulative_lt'],
      dtype='object')

In [62]:
# Convert 'month' and 'reqeust date' to datetime and remove time by normalization
df['month'] = pd.to_datetime(df['month'])
df['request_date'] = (pd.to_datetime(df['request_date'])).dt.normalize()

In [63]:
df_buy = df[df['make_or_buy_flag'] == 'Buy'].drop('make_or_buy_flag', 1)
df_make = df[df['make_or_buy_flag'] == 'Make'].drop('make_or_buy_flag', 1)